### 1. 최근접 이웃 협업 필터링


##### 협업 필터링 방식 중 하나인 최근접 이웃 협업 필터링에는 사용자 기반 최근접 이웃 방식과 아이템 기반 최근접 이웃 방식 두가지의 방법이 있습니다.

```

1-1) 사용자 기반 최근접 이웃 방식과 아이템 기반 최근접 이웃 방식에 대해 간단히 설명하시요.

```

```
답: 사용자 기반 최근접 이웃 방식은 특정 사용자와 유사한 다른 사용자의 데이터를 참고해서 추천해주는 방식이다.
아이템 기반 최근접 이웃 방식은 특정 사용자가 이용한 아이템 중 가징 비슷한 이용패턴을 보이는 아이템들을 추천해주는 방식이다. 예측하고자 하는 항목과 비슷한 항목들에 대하여 사용자가 높은 평가를 하였다면 그 항목도 높게 평가할 것이라고 예측하고, 낮은 평가를 하였다면 그 항목도 낮게 평가를 할 것이라고 예측하는 것이다. 
```

```
1-2) 사용자 기반 최근접 이웃 방식보다 아이템 기반 최근접 이웃 방식이 더 정확도가 높고 많이 사용하는 이유를 설명해 보시오.
```

```
답: 비슷한 상품을 좋아한다고 해서 사람들의 취향이 비슷하다고 판단하기는 어려운 경우가 많기 때문이다. 
```

### 2. 행렬분해

##### 2-1) 다음 행렬에 대해서 경사하강법을 이용한 행렬분해를 실습한 후 원본행렬과 비교해 보세요.

* 조건
```
K = 3
steps = 1000
learning_rate = 0.01
r_lambda = 0.01
```

In [3]:
import numpy as np

In [4]:
# 6X5 원본행렬

R = np.array([[1, 5, np.NaN, 2, np.NaN, np.NaN],
              [3, np.NaN, 1, 4, 3, np.NaN],
              [5, np.NaN, 2, 3, np.NaN, 1],
              [2, 5, np.NaN, 1, 2, 2],
              [3, 4, 2, np.NaN, 1, 1]])

In [7]:
num_users, num_items = R.shape
K = 3 #잠재요인의 개수(차원)

# P, Q를 임의의 값들을 가진 행렬로 초기화
np.random.seed(1)
P = np.random.normal(scale = 1./K, size = (num_users, K))
Q = np.random.normal(scale = 1./K, size = (num_items, K))

In [8]:
P

array([[ 0.54144845, -0.2039188 , -0.17605725],
       [-0.35765621,  0.28846921, -0.76717957],
       [ 0.58160392, -0.25373563,  0.10634637],
       [-0.08312346,  0.48736931, -0.68671357],
       [-0.1074724 , -0.12801812,  0.37792315]])

In [9]:
Q

array([[-0.36663042, -0.05747607, -0.29261947],
       [ 0.01407125,  0.19427174, -0.36687306],
       [ 0.38157457,  0.30053024,  0.16749811],
       [ 0.30028532, -0.22790929, -0.04096341],
       [-0.31192314, -0.08929603,  0.17678516],
       [-0.23055358, -0.13225118, -0.22905757]])

In [11]:
# 경사하강법이용 행렬분해

# 원본행렬에서 결측값이 아닌 값의 행, 열 , 실제 값을 튜플형태로 리스트안에 담기.
non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] >0]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01


for step in range(steps):
    for i, j, r in non_zeros:
        eij = r - np.dot(P[i, :], Q[j,:].T) # i번째 행, j번째 열 값의 오차.
        
        # L2정규화를 적용한 SGD기법 이용 P,Q 업데이트
        P[i,:] = P[i,:] + learning_rate * (eij * Q[j,:] - r_lambda * P[i,:])
        Q[j,:] = Q[j,:] + learning_rate * (eij * P[i,:] - r_lambda * Q[j,:])
    
    # 손실을 측정
    rmse = get_rmse(R, P, Q, non_zeros) # 매 스탭이 끝날 때마다 rmse값 산출
    
    if(step% 50) ==0:
        print('### iteration step: {} rmse: {}'.format(step, rmse))

### iteration step: 0 rmse: 2.8372963522394343
### iteration step: 50 rmse: 0.5869026473246871
### iteration step: 100 rmse: 0.4047660162403293
### iteration step: 150 rmse: 0.34082241791662643
### iteration step: 200 rmse: 0.3110742357172076
### iteration step: 250 rmse: 0.29103765829478123
### iteration step: 300 rmse: 0.27000330372920933
### iteration step: 350 rmse: 0.23908060330761152
### iteration step: 400 rmse: 0.1925346806524162
### iteration step: 450 rmse: 0.13741222983219675
### iteration step: 500 rmse: 0.09115237343020392
### iteration step: 550 rmse: 0.061572648855809625
### iteration step: 600 rmse: 0.04470139669657558
### iteration step: 650 rmse: 0.03506734892379227
### iteration step: 700 rmse: 0.029351672883474754
### iteration step: 750 rmse: 0.025838746576989827
### iteration step: 800 rmse: 0.023623813085577195
### iteration step: 850 rmse: 0.02220092002397485
### iteration step: 900 rmse: 0.02127328616636796
### iteration step: 950 rmse: 0.020661476731479018


In [12]:
pred_matrix = np.dot(P, Q.T)
print('예측행렬: \n', np.round(pred_matrix, 3))

예측행렬: 
 [[1.021 4.978 1.005 1.99  2.77  2.68 ]
 [2.995 5.862 1.001 3.969 2.982 2.802]
 [4.963 4.667 2.001 2.99  1.082 0.997]
 [1.979 4.992 1.983 1.029 1.979 1.97 ]
 [2.999 3.97  1.976 1.179 1.021 1.016]]


In [13]:
print(R)

[[ 1.  5. nan  2. nan nan]
 [ 3. nan  1.  4.  3. nan]
 [ 5. nan  2.  3. nan  1.]
 [ 2.  5. nan  1.  2.  2.]
 [ 3.  4.  2. nan  1.  1.]]


##### 2-2) 다음 RMSE를 계산하는 함수를 사용하여 오차를 확인하세요.

In [10]:
# RMSE
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    
    full_pred_matrix = np.dot(P, Q.T)
    
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [14]:
get_rmse(R, P, Q, non_zeros)

0.0202614677961177

### 3. 콘텐츠 기반 필터링

##### Book.csv 파일을 로드하고 콘텐츠 기반 필터링 방법을 사용하여 ㅇㅇ와 관련있는 10개의 책을 추천하는 실습입니다.

In [45]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

##### 3-1) Book.csv 파일을 로드하고 결측값이 있는 행을 제거하고 Genres 행렬을 단어의 나열로 전처리 한 후 Countvectorizer을 사용하여 백터행렬을 생성하시오.
* 조건</br>
CountVectorizer(min_df=0, ngram_range=(1,2))

In [46]:
data = pd.read_csv('Book.csv')
data.head()

,User-ID,Book-Rating,Book-Title,Book-Author,Publisher,genres
0,276725,3,Flesh Tones: A Novel,M. J. Rose,Ballantine Books,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
1,276726,5,Rites of Passage,Judith Rae,Heinle,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""..."
2,276727,4,The Notebook,Nicholas Sparks,Warner Books,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
3,276729,3,Help!: Level 1,Philip Prowse,Cambridge University Press,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam..."
4,276729,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,Cambridge University Press,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."


In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
User-ID        500 non-null int64
Book-Rating    500 non-null int64
Book-Title     370 non-null object
Book-Author    370 non-null object
Publisher      370 non-null object
genres         500 non-null object
dtypes: int64(2), object(4)
memory usage: 23.5+ KB


In [60]:
book_df = data.dropna(axis=0)

In [61]:
book_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 370 entries, 0 to 499
Data columns (total 6 columns):
User-ID        370 non-null int64
Book-Rating    370 non-null int64
Book-Title     370 non-null object
Book-Author    370 non-null object
Publisher      370 non-null object
genres         370 non-null object
dtypes: int64(2), object(4)
memory usage: 20.2+ KB


In [62]:
# genres 전처리
book_df['genres'].head()

0    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1    [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
Name: genres, dtype: object

In [63]:
from ast import literal_eval
# 문자열을 이 문자열이 의미하는 list [dict1, dict2] 객체로 만들 수 있다.

book_df['genres'] = book_df['genres'].apply(literal_eval)
book_df['genres'] = book_df['genres'].apply(lambda x : [y['name'] for y in x])
book_df['genres_literal'] = book_df['genres'].apply(lambda x: (' ').join(x))
book_df['genres_literal'].head()

0    Action Adventure Fantasy Science Fiction
1                    Adventure Fantasy Action
2                      Action Adventure Crime
3                 Action Crime Drama Thriller
4            Action Adventure Science Fiction
Name: genres_literal, dtype: object

In [66]:
# Countvectorizer을 사용하여 백터행렬을 생성

from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df =0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(book_df['genres_literal'])

genre_mat.shape

(370, 147)

##### 3-2) 생성된 백터 행렬으로 코사인 유사도를 계산하고 행렬로 변환한 뒤 유사도가 높은 순으로 정렬하시오.

In [72]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
genre_sim[:4]

array([[1.        , 0.59628479, 0.4472136 , ..., 0.62994079, 0.        ,
        0.        ],
       [0.59628479, 1.        , 0.4       , ..., 0.3380617 , 0.        ,
        0.        ],
       [0.4472136 , 0.4       , 1.        , ..., 0.3380617 , 0.        ,
        0.        ],
       [0.12598816, 0.16903085, 0.3380617 , ..., 0.14285714, 0.        ,
        0.        ]])

In [97]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
genre_sim_sorted_ind

array([[  0,  11,  37, ..., 221, 222, 369],
       [ 72,   9,   1, ...,  55, 323, 369],
       [  2, 268, 351, ..., 134, 137, 369],
       ...,
       [184, 367, 288, ...,  79, 185, 369],
       [368,  34, 141, ..., 195, 194,   0],
       [369, 311, 254, ..., 215, 216,   0]], dtype=int64)

##### 3-3) 장르 유사도에 따라 영화를 추천하는 다음 함수의 '빈칸'을 채우고 'The Notebook'이라는 책을 좋아하는 사용자에게 보여줄 추천 결과를 출력하시오(10개).

In [93]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    title_movie = df[df['Book-Title'] == title_name]
    
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [98]:
similar_movies = find_sim_movie(book_df, genre_sim_sorted_ind, 'The Notebook', 10)
similar_movies[['Book-Title', 'Book-Rating', 'Book-Author']]

[[  2 268 351  18 237  46 297  17  24  32]]


,Book-Title,Book-Rating,Book-Author
2,The Notebook,4,Nicholas Sparks
356,Oceano Mare,6,Alessandro Baricco
469,Contacto,6,Carl Sagan
21,How to Deal With Difficult People,7,Rick Brinkman
304,Verraten und verkauft. Roman.,3,Philippe Djian
62,Henry der Held.,10,Roddy Doyle
411,The Tower at Stony Wood,8,Patricia A. McKillip
20,Birdsong: A Novel of Love and War,8,Sebastian Faulks
29,The Pillars of the Earth,5,Ken Follett
38,Besiegt vom Sturm der Leidenschaft.,4,Patricia Potter
